In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# sparkSession = spark --> in the case of EMR
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

In [ ]:
from pyspark.sql.types import *

custom_schema = StructType([
    StructField("dt_server", StringType()),
    StructField("dt_avl", StringType()),
    StructField("line_id", IntegerType()),
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("id_avl", IntegerType()),
    StructField("event", IntegerType()),
    StructField("id_point", IntegerType()),
    StructField("hour", IntegerType()),
    StructField("hour_diff", FloatType())
])

In [ ]:
# hours between 06:00 and 22:59:59
for day in range(1,32):
    traces = spark.read.csv("s3a://mobility-traces-sp/raw-hour-and-hour-diff/MO_1510"+ str(day) + "/", header = 'false',schema=custom_schema)
    traces_filtered = traces.filter('hour in (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22)')
    traces_filtered.repartition(60).write.parquet("s3://mobility-traces-sp/processed-data/records-between-6-23-hour/MO_1510"+ str(day) + "/")